In [ ]:
#do not run --initial run only
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("another-session")
    # Delta
    .config("spark.jars.packages", "io.delta:delta-spark_2.13:4.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # Hive support
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", "file:/Users/Binaya/Documents/spark1/spark-warehouse")
    .config(
        "javax.jdo.option.ConnectionURL",
        "jdbc:derby:;databaseName=/Users/Binaya/Documents/spark1/metastore_db;create=true"
    )
    .enableHiveSupport()
    .getOrCreate()
)

In [ ]:
spark.sql("""
          select count(*) from retail_raw.sales_raw
          """).show()

In [ ]:
print('=== dim_employee ===')
spark.sql(""" select count(*) from retail_gold.fact_sales  """).show(truncate=False)

In [ ]:
print('=== dim_employee ===')
spark.sql(""" select * from retail_gold.dim_employee """).show(truncate=False)

In [ ]:
from pyspark.sql.functions import *
print('==dim_customer==')
spark.sql(""" select * from retail_gold.dim_customer""").show()

print('=== dim_employee ===')
spark.sql(""" select * from retail_gold.dim_employee """).show()

print('==dim_product==')
spark.sql(""" select * from retail_gold.dim_product""").show(truncate=False)

print('==dim_store==')
spark.sql(""" select * from retail_gold.dim_store""").show(truncate=False)

print('==dim_supplier==')
spark.sql(""" select * from retail_gold.dim_supplier""").show(truncate=False)

print('==dim_date==')
spark.sql(""" select * from retail_gold.dim_date""").show(truncate=False)

In [ ]:
print('=== Fact Table ===')

spark.sql(""" select distinct
          sale_id, date_sk, customer_sk, product_sk, store_sk, quantity, unit_price, discount_amt, gross_amt, net_amt, load_ts 
          from retail_gold.fact_sales 
         where employee_sk is not null
           """).show(truncate=False)

In [ ]:
spark.sql("""
SELECT
          distinct
e.employee_id , s.employee_id 
FROM retail_silver.sales_clean s
LEFT JOIN retail_gold.dim_date     d  ON d.full_date = CAST(s.sale_ts AS DATE)
LEFT JOIN retail_gold.dim_product  p  ON p.product_id = s.product_id
LEFT JOIN retail_gold.dim_store    st ON st.store_id = s.store_id
LEFT JOIN retail_gold.dim_customer c  ON c.customer_id = s.customer_id AND c.is_current = true
LEFT JOIN retail_gold.dim_employee e  ON e.employee_id = s.employee_id AND e.is_current = true
          """).show(10000)

In [ ]:
spark.sql(""" select distinct _c5 from csv.`data/sales/sales.csv` order by 1 desc""").show(1000)